In [84]:
import tensorflow as tf
print(tf.__version__)
tf.test.is_gpu_available()

2.7.0


True

In [85]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15046188559648542828
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6165991424
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3743705056808693554
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:0a:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [86]:
import numpy as np
import pandas as pd

from math import ceil
from numba import njit, prange
from sklearn.utils.validation import check_array
from pyts.preprocessing import MinMaxScaler
from pyts.approximation import PiecewiseAggregateApproximation
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import os 
import cv2

In [87]:
Welding_data = np.load('E:/Result/ver.3.22/MTF/MTF.npz')

X_data = Welding_data['X_data']
y_data = Welding_data['y_data']
i_data = Welding_data['i_data']

Welding_data.close()

In [88]:
seed = 7
np.random.seed()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test,i_train, i_test = train_test_split(X_data, y_data, i_data, test_size= 0.2, shuffle= True, random_state= seed)

In [89]:
inputs = np.concatenate((X_train, X_test))
targets = np.concatenate((y_train, y_test))
index = np.concatenate((i_train, i_test))

In [90]:
targets = np_utils.to_categorical(targets)

In [93]:
from keras.applications.efficientnet import EfficientNetB0
from keras.layers import Dense, Input, Activation, Flatten, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization.batch_normalization import BatchNormalization

In [94]:
input = Input(shape= (300,300,2))
model = EfficientNetB0(
    input_tensor= input,
    include_top= False,
    weights= None,
    pooling= 'avg'
)

x = model.output
x = Dropout(0,2)(x)
x = Dense(3, activation= 'softmax', name= 'softmax')(x)

model = Model(model.input, x)
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 300, 300, 2  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_9 (Rescaling)        (None, 300, 300, 2)  0           ['input_10[0][0]']               
                                                                                                  
 normalization_9 (Normalization  (None, 300, 300, 2)  5          ['rescaling_9[0][0]']            
 )                                                                                                
                                                                                            

In [95]:
#tf.keras.utils.plot_model(model, show_shapes= True)

In [96]:
import keras.backend as K
import datetime

log_dir = "logs_EffiB1/my_board/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= log_dir, histogram_freq= 1)

In [97]:

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kfold = KFold(n_splits=10, shuffle= True, random_state= seed)

test= []
train= []
test_= []
train_= []

for train, test in kfold.split(inputs, targets):
    
    print(len(test))
    print(len(train))
    
    for i in zip(test):
        test_.append(i)
    for i in zip(train):
        train_.append(i)


118
1061
118
1061
118
1061
118
1061
118
1061
118
1061
118
1061
118
1061
118
1061
117
1062


In [98]:
train = train_[0:1061]
train = np.reshape(train, 1061)
test = test_[0:117]
test = np.reshape(test, 117)

In [99]:
print(inputs.shape)
print(targets.shape)
print(index.shape)

(1179, 300, 300, 2)
(1179, 3)
(1179,)


In [100]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

acc_per_fold = []
loss_per_fold = []

kfold = StratifiedKFold(n_splits=10, shuffle=True)

fold_no = 1

with tf.device('/gpu:0'):
    for train, test in kfold.split(X_train, y_train):
        print(train.shape, test.shape)
    
        input = Input(shape=(300, 300, 2))
        model = EfficientNetB0(input_tensor=input, include_top=False, weights=None, pooling='avg')
        
        x = model.output

        x = Dense(3, activation='softmax', name='softmax', kernel_initializer='he_normal')(x)
        model = Model(model.input, x)

        
        optimizer = tf.keras.optimizers.SGD(lr=0.01, decay=1e-3, momentum=0.9, nesterov=True)
        #optimizer = optimizers.Adam(lr=0.001)
        
        #callbacks_list = [LearningRateSchedule([20,40])]
        
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        
        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')
        
        history = model.fit(inputs[train], targets[train], 
                            batch_size=16, 
                            epochs=20, 
                            verbose=1,
                            validation_data=(inputs[test], targets[test]),
                            callbacks = tensorboard_callback) # 여기에 Validation set을 넣어야되는거 아닌가?
        
        scores = model.evaluate(inputs[test], targets[test], verbose=0)
        print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
        acc_per_fold.append(scores[1] * 100)
        loss_per_fold.append(scores[0])
     

        np.savetxt('E:/Result/ver.3.25/EfficientNetB1/acc/'+ f'acc_{i}.csv', acc_per_fold, delimiter=",")
        np.savetxt('E:/Result/ver.3.25/EfficientNetB1/loss/'+ f'loss_{i}.csv', loss_per_fold, delimiter=",")
        
        model.save('E:/Result/ver.3.25/EfficientNetB1/' + f'Eff_{fold_no}.h5',fold_no)
        
        fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')


(848,) (95,)


C:\Users\jwhyu\anaconda3\envs\venv\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


------------------------------------------------------------------------
Training for fold 1 ...


C:\Users\jwhyu\anaconda3\envs\venv\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/20
53/53 [==============================] - 19s 263ms/step - loss: 0.6096 - accuracy: 0.4717 - val_loss: 0.6746 - val_accuracy: 0.3474
Epoch 2/20
53/53 [==============================] - 13s 240ms/step - loss: 0.4928 - accuracy: 0.6026 - val_loss: 0.7637 - val_accuracy: 0.3474
Epoch 3/20
53/53 [==============================] - 13s 240ms/step - loss: 0.4061 - accuracy: 0.6946 - val_loss: 0.8583 - val_accuracy: 0.3474
Epoch 4/20
53/53 [==============================] - 13s 238ms/step - loss: 0.3465 - accuracy: 0.7488 - val_loss: 1.1203 - val_accuracy: 0.3474
Epoch 5/20
53/53 [==============================] - 13s 238ms/step - loss: 0.3154 - accuracy: 0.8066 - val_loss: 0.8377 - val_accuracy: 0.3474
Epoch 6/20
53/53 [==============================] - 13s 239ms/step - loss: 0.2482 - accuracy: 0.8432 - val_loss: 0.8519 - val_accuracy: 0.3474
Epoch 7/20
53/53 [==============================] - 13s 237ms/step - loss: 0.2235 - accuracy: 0.8715 - val_loss: 1.0440 - val_accuracy: 0.3263

FileNotFoundError: [Errno 2] No such file or directory: 'E:/Result/ver.3.25/EfficientNetB1/acc/acc_(1178,).csv'

In [ ]:
import matplotlib.pylab as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training', 'validation'], loc= 'upper right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validaion'], loc= 'upper right')
plt.show()